<a href="https://colab.research.google.com/github/tgusmao/programacao_ds/blob/master/TrabalhoPratico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Prático de Programação para Ciências de Dados**

Dupla: 
*   Tammy Hikari Yanai Gusmão
*   Thalita Naiara Alves


Trabalho Prático para obtenção de nota na disciplina de Programação para Ciências de Dados - Turma 1 de 2020.

##Importação da Base de Dados

In [ ]:
import pandas as pd

Iniciando abaixo a leitura do arquivo CSV local (localizado no Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_original = pd.read_csv('/content/drive/My Drive/dados-curso-completo.csv', sep = ',', skiprows = 0, encoding = 'utf-8')

In [ ]:
#cópia da base original
df_copia_original = df_original.copy()

In [ ]:
#criação de uma coluna extra, para armazenar o idioma correto dos tweets
df_original['idioma_detectado'] = ''

##Resumo do dataset original



A base de dados contém informações sobre tuítes sobre a COVID-19, postados durante o primeiro semestre do ano de 2020. A base foi levemente pré-processada anteriormente pelo professor da disciplina.  

In [ ]:
df_original.shape

Como pode ser visto acima, o atributo *shape* retornou uma tupla contendo a quantidade de linhas e colunas que constituem a base: 1.658.825 linhas e 13 colunas. 

In [ ]:
df_original.columns

As 13 colunas foram identificadas pelos seguintes nomes: 
data, usuario (nome de perfil), apelido 

*   data;
*   usuario (nome de perfil);
*   apelido (username)
*   texto (tuíte em si);
*   retweet;
*   seguidores;
*   idioma;
*   lugar;
*   pais;
*   sigla;
*   latitude;
*   longitude;
*   hashtags.

In [ ]:
df_original.isnull().sum()

data                0
usuario            57
apelido             0
texto               0
retweet             0
seguidores          0
idioma              0
lugar         1584220
pais          1584225
sigla         1584230
latitude      1650105
longitude     1650105
hashtags            0
dtype: int64

## Tratamento de Dados

De acordo com os achados apresentados anteriormente, as colunas *usuario*, *lugar*, *pais* e *sigla*. Por conta da natureza dos atributos, *latitude* e *longitude* serão mantidos como estão para uma análise posterior.

A seguir, temos o preenchimento de dados aplicados somente nas colunas *usuario*, *lugar*, *pais* e *sigla*. 

In [ ]:
df_original['usuario'] = df_original['usuario'].fillna('')
df_original['lugar'] = df_original['lugar'].fillna('Não Informado')
df_original['pais'] = df_original['pais'].fillna('Não Informado')
df_original['sigla'] = df_original['sigla'].fillna('Não Informado')

Em uma busca manual, foi conferido que existem tweets repetidos. Para tanto, foi aplicada a função de remoção de linhas duplicadas.

In [ ]:
df_reduzida = pd.DataFrame.drop_duplicates(df_original)
print(df_total.shape)
print(df_reduzida.shape)

A remoção da linhas duplicadas mostra que haviam 3 tweets iguais no dataset. Somente uma delas permaneceu no dataset.

In [ ]:
df_original = df_original.rename(columns = {'Data':'data','Usuario':'nome_perfil', 'Apelido':'username', 'Texto':'tweet', 'Retweet':'qntd_retweet', 'Seguidores':'qntd_seguidores', 'Idioma':'idioma', 'Lugar':'lugar', 'Pais':'pais', 'Sigla':'sigla_pais', 'Latitude':'lat', 'Longitude':'long', 'Hashtags':'hashtags'})

##Classificação de Temas